 This notebook is adapted from: 

 "Data Cleaning" by Shahriar Kabir

 Source: https://www.kaggle.com/code/shahriarkabir/data-cleaning#Data-Cleaning-Order

 Licensed under the Apache License, Version 2.0 (http://www.apache.org/licenses/LICENSE-2.0)
 
 Modifications: reduced comments to get the cleaning


In [62]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [63]:
# load dataset
df = pd.read_csv('laptopData.csv')

In [64]:
# remove Unnamed: 0 column from this dataset 
# cause this column is not important 
df.drop('Unnamed: 0', axis=1, inplace=True)

# Issues with the dataset

**`Dirty Data`**


- 30 rows out of 1303 are totally missing along with their 11 columns, the primary data quality issue is a `completeness`.

- 58 duplicated rows detected, with 30 rows containing all null values, indicating potential data entry errors or missing information.`completeness`


- **`Company`:**

- The "Company" column contains values that are expected to represent model names, with "Acer" being an example, but instead includes "Vero,"`consistency`

- **`TypeName`:**

- To correct the misspelled value "notbook" to "notebook" in the "TypeName" column.`consistency`
- Inconsistent data type for the "TypeName" column.`consistency`

- **`Inches`:**

- Convert all centimeter (cm) values to inches for consistency in measurement units.`consistency`

- The presence of '?' values in the "inches" column indicates missing or invalid data. `Validity`

- all Ichces column that are >= 24 need to verify manually `consistency`

- **`Ram`**

- Cleanse the RAM column to retain only numerical values, excluding the unit 'GB', for improved data consistency and analysis.`consistency`

- Some inconsistencies are observed in the RAM sizes listed, which do not align with other features of the laptops. Manual verification through reliable sources such as Google is necessary to rectify these discrepancies and ensure accurate representation of RAM specifications.`accuracy`

- **`Memory`**

- resolving the '?' entries in the memory column `completeness`

- **`Gpu`**
- Extracting only GPU brand from the 'Gpu' column may lead to potential data loss of specific GPU model information, impacting the understanding of GPU performance relative to laptop prices.`accuracy`

- **`OpSys`**
- Inconsistency in mentioning "Android" as the operating system for a Lenovo 2-in-1 convertible device raises accuracy concerns in the dataset.`consistency`

- The issue in the OS column some os values is inconsistency in operating system versions, leading to potential data quality problems.`consistency`

- **`Weight`**
- there is an inconsistency where one laptop's weight is listed as 0.0002kg, which seems implausible and raises concerns about data accuracy.`validity`

- The recorded weights for Acer, Toshiba, and Lenovo laptops are notably higher than anticipated. Upon manual verification via Google, it was discovered that the actual weights are considerably lower, suggesting potential inaccuracies in the dataset. The corrected weights for Acer, Toshiba, and Lenovo laptops should be 2.2kg, 1.39kg, and 1.27kg, respectively.`validity`

**`Messy Data`**

- **`ScreenResolution`**
- Convert "ScreenResolution" column into multiple columns to separate resolution dimensions and display type for improved data structure and analysis.

- **`Cpu`**
- Optimize data structure and analysis by transforming the "CPU" column into multiple columns, facilitating accurate extraction of processor brand, type, and speed.

- **`Memory`**

- The memory column exhibits structural inconsistencies, necessitating transformation into two separate columns for SSD and HDD memory sizes. This restructuring will enhance data clarity and facilitate more precise analysis of storage specifications across laptops.

In [65]:
# copy dataframe as tem_df
tem_df = df.copy()

In [66]:
# unique tem_df Inches columns
tem_df['Inches'].unique()

array(['13.3', '15.6', '15.4', '14', '12', nan, '17.3', '13.5', '12.5',
       '13', '18.4', '13.9', '11.6', '25.6', '35.6', '12.3', '27.3', '24',
       '33.5', '?', '31.6', '17', '15', '14.1', '11.3', '10.1'],
      dtype=object)

In [67]:
tem_df['ScreenResolution'].str.split('HD').str.get(0).str.split('Display').str.get(0).value_counts()

ScreenResolution
Full                                 540
IPS Panel Full                       283
1366x768                             274
IPS Panel 4K Ultra                    23
1600x900                              23
Quad                                  17
IPS Panel Retina                      17
4K Ultra                              16
Touchscreen 1366x768                  16
IPS Panel Quad                        11
IPS Panel 1366x768                     7
Touchscreen 2560x1440                  6
Touchscreen 2256x1504                  6
IPS Panel Touchscreen 2560x1440        5
IPS Panel 2560x1440                    4
1440x900                               4
2560x1440                              3
Touchscreen 2400x1600                  3
1920x1080                              3
IPS Panel Touchscreen 1366x768         3
IPS Panel Touchscreen 1920x1200        3
IPS Panel Touchscreen / 4K Ultra       2
Touchscreen / Quad                     1
Touchscreen / Full                     1

In [68]:
tem_df['ScreenResolution'].str.extract(r'([A-Za-z\s]+) Panel').value_counts(dropna=False)

0  
NaN    944
IPS    359
Name: count, dtype: int64

In [69]:
# Regular expression pattern to find resolutions
pattern = r'\b\d{3,4}x\d{3,4}\b'

# Find resolutions for each value in the Series
tem_df['ScreenResolution'].str.findall(pattern).str.get(0).value_counts().index

Index(['1920x1080', '1366x768', '3840x2160', '3200x1800', '1600x900',
       '2560x1440', '2304x1440', '2560x1600', '2256x1504', '2880x1800',
       '1440x900', '1920x1200', '2400x1600', '2160x1440', '2736x1824'],
      dtype='object', name='ScreenResolution')

In [70]:
tem_df[tem_df['ScreenResolution'].fillna('').str.contains('Full HD', case=False)]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
8,Asus,Ultrabook,14,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,512GB SSD,Nvidia GeForce MX150,Windows 10,1.3kg,79653.6000
9,Acer,Ultrabook,14,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Intel UHD Graphics 620,Windows 10,1.6kg,41025.6000
11,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,500GB HDD,Intel HD Graphics 520,No OS,1.86kg,18381.0672
13,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,256GB SSD,AMD Radeon R5 M430,Windows 10,2.2kg,26581.3920
...,...,...,...,...,...,...,...,...,...,...,...
1284,Lenovo,2 in 1 Convertible,14,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,33992.6400
1289,Asus,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core M 6Y30 0.9GHz,8GB,512GB SSD,Intel HD Graphics 515,Windows 10,1.2kg,38841.1200
1293,Lenovo,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2.6kg,47898.7200
1294,HP,Notebook,15.6,Full HD 1920x1080,AMD A9-Series 9410 2.9GHz,6GB,1.0TB Hybrid,AMD Radeon R7 M440,Windows 10,2.04kg,29303.4672


#### Explore the 'Cpu' column and identify any inconsistencies.

In [71]:
tem_df['Cpu'].unique()

array(['Intel Core i5 2.3GHz', 'Intel Core i5 1.8GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Core i7 2.7GHz',
       'Intel Core i5 3.1GHz', 'AMD A9-Series 9420 3GHz',
       'Intel Core i7 2.2GHz', 'Intel Core i7 8550U 1.8GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Core i7 2.8GHz', 'Intel Core M m3 1.2GHz',
       'Intel Core i7 7500U 2.7GHz', 'Intel Core i7 2.9GHz',
       'Intel Core i3 7100U 2.4GHz', nan, 'Intel Core i5 7300HQ 2.5GHz',
       'AMD E-Series E2-9000e 1.5GHz', 'Intel Core i5 1.6GHz',
       'Intel Core i7 8650U 1.9GHz', 'Intel Atom x5-Z8300 1.44GHz',
       'AMD E-Series E2-6110 1.5GHz', 'AMD A6-Series 9220 2.5GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 2.0GHz', 'AMD Ryzen 1700 3GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Celeron Dual Core N3060 1.6GHz', 'Intel Core i5 1.3GHz',
       'AMD FX 9830P 3GH

In [72]:
# Define regex pattern to match any number followed by "GHz"
pattern = r'\b\d+(?:\.\d+)?(?:GHz|Hz)\b'

# Find all matches in the Processor column
tem_df['Cpu'].str.findall(pattern).value_counts()


Cpu
[2.5GHz]     283
[2.8GHz]     161
[2.7GHz]     159
[1.6GHz]     128
[2.3GHz]      84
[1.8GHz]      77
[2.6GHz]      75
[2GHz]        65
[1.1GHz]      53
[2.4GHz]      52
[2.9GHz]      21
[3GHz]        19
[2.0GHz]      19
[1.2GHz]      15
[2.2GHz]      11
[1.44GHz]     10
[1.5GHz]      10
[1.3GHz]       6
[3.6GHz]       5
[0.9GHz]       4
[3.1GHz]       3
[2.50GHz]      3
[2.1GHz]       3
[1.9GHz]       2
[1.60GHz]      1
[1.0GHz]       1
[3.2GHz]       1
[1.92GHz]      1
[2.70GHz]      1
Name: count, dtype: int64

#### Explore the 'Ram' column and identify any inconsistencies

In [73]:
tem_df['Ram'].value_counts().index

Index(['8GB', '4GB', '16GB', '6GB', '12GB', '2GB', '32GB', '64GB', '24GB',
       '1GB'],
      dtype='object', name='Ram')

In [74]:
## in first two rows that ram size should be 8gb according to google
tem_df[tem_df['Ram'] == '64GB']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
71,Dell,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,64GB,256GB SSD,AMD Radeon 530,Windows 10,1.4kg,50882.4000
720,Lenovo,Ultrabook,14,IPS Panel Quad HD+ 2560x1440,Intel Core i7 6500U 2.5GHz,64GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.17kg,89864.1792
1066,Asus,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 6820HK 2.7GHz,64GB,1TB SSD,Nvidia GeForce GTX 980,Windows 10,3.58kg,211788.0000


In [75]:
tem_df[tem_df['Ram'] == '24GB']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
438,Lenovo,Ultrabook,24,2560x1440,Intel Core i7 7500U 2.7GHz,24GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.32kg,126912.96
494,Asus,Gaming,17.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6700HQ 2.6GHz,24GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 965M,Windows 10,4.33kg,67612.32
517,Asus,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,24GB,512GB SSD,Nvidia GeForce GTX1080,Windows 10,2.24kg,158135.04


In [76]:
tem_df[tem_df['Ram'] == '2GB']

# in acer notebook full-hd display in ram should 8 gb
# and mediacom laptop ram might be 4gb

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
31,Asus,Notebook,14,1366x768,AMD E-Series E2-6110 1.5GHz,2GB,32GB Flash Storage,AMD Radeon R2,Windows 10,1.65kg,10602.720
91,Acer,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,2GB,1TB HDD,Nvidia GeForce MX150,Windows 10,2.2kg,36709.920
290,Acer,Notebook,15.6,1366x768,Intel Celeron Dual Core 3205U 1.5GHz,2GB,16GB SSD,Intel HD Graphics,Chrome OS,2.19kg,10602.720
314,Asus,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg,14652.000
346,HP,Notebook,14,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2GB,32GB Flash Storage,Intel HD Graphics 400,Windows 10,1.44kg,14865.120
409,Lenovo,Notebook,11.6,1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.15kg,16221.096
515,Asus,Netbook,11.6,1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg,13053.600
626,Mediacom,Notebook,14,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg,12733.920
637,Lenovo,Notebook,14,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2GB,32GB Flash Storage,Intel HD Graphics 400,Windows 10,1.42kg,13266.720
745,Acer,Notebook,14,1366x768,Intel Celeron Dual Core N3060 1.6GHz,2GB,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.68kg,17582.400


In [77]:
tem_df[tem_df['Ram'] == '16GB'].sample(2)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
945,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,AMD FX 8800P 2.1GHz,16GB,512GB SSD + 1TB HDD,AMD Radeon R9 M385,Windows 10,2.5kg,53226.72
1209,Asus,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.2kg,130482.72


In [78]:
tem_df[tem_df['Ram'] == '12GB'].sample(2)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
428,HP,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,12GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,3.35kg,106506.72
588,Lenovo,Notebook,15.6,Touchscreen 1366x768,Intel Core i7 8550U 1.8GHz,12GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.2kg,32447.52


In [79]:
tem_df[tem_df['Ram'] == '6GB'].sample(3)

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
572,HP,Notebook,15.6,IPS Panel Full HD 1920x1080,AMD A9-Series A9-9420 3GHz,6GB,256GB SSD,AMD Radeon 530,Windows 10,1.95kg,34578.72
120,Acer,Notebook,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i3 7100U 2.4GHz,6GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.1kg,25521.12
982,Lenovo,Notebook,15.6,Full HD 1920x1080,AMD A12-Series 9720P 3.6GHz,6GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,29250.72


In [80]:
tem_df[tem_df['Ram'] == '1GB']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
706,Acer,2 in 1 Convertible,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 8250U 1.6GHz,1GB,256GB SSD,Intel UHD Graphics 620,Windows 10,1.6kg,53226.72


where ram size is 1gb i check manully in google that model should be 8gb 
according to model specifications 

#### Explore the 'Memory' column and identify any inconsistencies

In [81]:
tem_df['Memory'].value_counts().index

Index(['256GB SSD', '1TB HDD', '500GB HDD', '512GB SSD',
       '128GB SSD +  1TB HDD', '128GB SSD', '256GB SSD +  1TB HDD',
       '32GB Flash Storage', '2TB HDD', '64GB Flash Storage',
       '512GB SSD +  1TB HDD', '1TB SSD', '256GB SSD +  2TB HDD',
       '1.0TB Hybrid', '256GB Flash Storage', '16GB Flash Storage', '32GB SSD',
       '128GB Flash Storage', '180GB SSD', '16GB SSD', '512GB SSD +  2TB HDD',
       '256GB SSD +  500GB HDD', '128GB SSD +  2TB HDD',
       '256GB SSD +  256GB SSD', '1TB SSD +  1TB HDD', '512GB Flash Storage',
       '512GB SSD +  512GB SSD', '64GB SSD', '64GB Flash Storage +  1TB HDD',
       '1TB HDD +  1TB HDD', '32GB HDD', '512GB SSD +  256GB SSD', '?',
       '128GB HDD', '240GB SSD', '8GB SSD', '508GB Hybrid', '1.0TB HDD',
       '512GB SSD +  1.0TB Hybrid', '256GB SSD +  1.0TB Hybrid'],
      dtype='object', name='Memory')

In [82]:
tem_df['Memory'].value_counts()

Memory
256GB SSD                        401
1TB HDD                          217
500GB HDD                        130
512GB SSD                        116
128GB SSD +  1TB HDD              92
128GB SSD                         74
256GB SSD +  1TB HDD              71
32GB Flash Storage                37
2TB HDD                           16
64GB Flash Storage                14
512GB SSD +  1TB HDD              14
1TB SSD                           13
256GB SSD +  2TB HDD              10
1.0TB Hybrid                       9
256GB Flash Storage                8
16GB Flash Storage                 7
32GB SSD                           6
128GB Flash Storage                4
180GB SSD                          4
16GB SSD                           3
512GB SSD +  2TB HDD               3
256GB SSD +  500GB HDD             2
128GB SSD +  2TB HDD               2
256GB SSD +  256GB SSD             2
1TB SSD +  1TB HDD                 2
512GB Flash Storage                2
512GB SSD +  512GB SSD         

In [83]:
tem_df.loc[[350, 447, 795, 1083]]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,Dell,Workstation,15.6,Full HD 1920x1080,Intel Xeon E3-1505M V6 3GHz,8GB,64GB Flash Storage + 1TB HDD,Nvidia Quadro M620,Windows 10,2.23kg,106187.04
795,Lenovo,2 in 1 Convertible,14,IPS Panel Touchscreen 2560x1440,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.42kg,124142.40
1083,HP,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.3kg,109170.72


In [84]:
tem_df.loc[770:775]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
770,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,?,AMD Radeon R7 M445,Windows 10,2.3kg,62938.0656
771,Samsung,2 in 1 Convertible,15,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,256GB SSD,AMD Radeon 540,Windows 10,1.71kg,95850.7200
772,Toshiba,Notebook,14,IPS Panel 1366x768,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.75kg,54345.6000
773,Dell,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.62kg,58554.7200
774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
775,Asus,2 in 1 Convertible,15.6,Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,12GB,2TB HDD,Nvidia GeForce 940MX,Windows 10,2.3kg,44701.9200


#### Explore the 'Gpu' column and identify any inconsistencies

In [85]:
tem_df['Gpu'].str.split(' ').str.get(0).value_counts()

Gpu
Intel     703
Nvidia    393
AMD       176
ARM         1
Name: count, dtype: int64

In [86]:
tem_df['Gpu'].unique()

array(['Intel Iris Plus Graphics 640', 'Intel HD Graphics 6000',
       'Intel HD Graphics 620', 'AMD Radeon Pro 455',
       'Intel Iris Plus Graphics 650', 'AMD Radeon R5',
       'Intel Iris Pro Graphics', 'Nvidia GeForce MX150',
       'Intel UHD Graphics 620', 'Intel HD Graphics 520',
       'AMD Radeon Pro 555', 'AMD Radeon R5 M430',
       'Intel HD Graphics 615', 'AMD Radeon Pro 560',
       'Nvidia GeForce 940MX', nan, 'Nvidia GeForce GTX 1050',
       'AMD Radeon R2', 'AMD Radeon 530', 'Nvidia GeForce 930MX',
       'Intel HD Graphics', 'Intel HD Graphics 500',
       'Nvidia GeForce 930MX ', 'Nvidia GeForce GTX 1060',
       'Nvidia GeForce 150MX', 'Intel Iris Graphics 540',
       'AMD Radeon RX 580', 'Nvidia GeForce 920MX',
       'AMD Radeon R4 Graphics', 'AMD Radeon 520',
       'Nvidia GeForce GTX 1070', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 400', 'Nvidia GeForce MX130', 'AMD R4 Graphics',
       'Nvidia GeForce GTX 940MX', 'AMD Radeon RX 560',
       

In [87]:
tem_df[tem_df['Gpu'].fillna('').str.contains('ARM')]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
1191,Samsung,2 in 1 Convertible,12.3,IPS Panel Touchscreen 2400x1600,Samsung Cortex A72&A53 2.0GHz,4GB,32GB Flash Storage,ARM Mali T860 MP4,Chrome OS,1.15kg,35111.52


In [88]:
tem_df['Gpu'].unique()

array(['Intel Iris Plus Graphics 640', 'Intel HD Graphics 6000',
       'Intel HD Graphics 620', 'AMD Radeon Pro 455',
       'Intel Iris Plus Graphics 650', 'AMD Radeon R5',
       'Intel Iris Pro Graphics', 'Nvidia GeForce MX150',
       'Intel UHD Graphics 620', 'Intel HD Graphics 520',
       'AMD Radeon Pro 555', 'AMD Radeon R5 M430',
       'Intel HD Graphics 615', 'AMD Radeon Pro 560',
       'Nvidia GeForce 940MX', nan, 'Nvidia GeForce GTX 1050',
       'AMD Radeon R2', 'AMD Radeon 530', 'Nvidia GeForce 930MX',
       'Intel HD Graphics', 'Intel HD Graphics 500',
       'Nvidia GeForce 930MX ', 'Nvidia GeForce GTX 1060',
       'Nvidia GeForce 150MX', 'Intel Iris Graphics 540',
       'AMD Radeon RX 580', 'Nvidia GeForce 920MX',
       'AMD Radeon R4 Graphics', 'AMD Radeon 520',
       'Nvidia GeForce GTX 1070', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 400', 'Nvidia GeForce MX130', 'AMD R4 Graphics',
       'Nvidia GeForce GTX 940MX', 'AMD Radeon RX 560',
       

#### Explore the 'OpSys' column and identify any inconsistencies

In [89]:
tem_df['OpSys'].value_counts()

OpSys
Windows 10      1047
No OS             63
Linux             61
Windows 7         45
Chrome OS         27
macOS             13
Mac OS X           8
Windows 10 S       8
Android            1
Name: count, dtype: int64

In [90]:
tem_df[tem_df['OpSys'] == 'Android']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
1114,Lenovo,2 in 1 Convertible,10.1,IPS Panel Touchscreen 1920x1200,Intel Atom x5-Z8550 1.44GHz,4GB,64GB Flash Storage,Intel HD Graphics 400,Android,0.69kg,29250.72


- Inconsistency in mentioning "Android" as the operating system for a Lenovo 2-in-1 convertible device raises accuracy concerns in the dataset.`consistency`

- The issue in the OS column some os values is inconsistency in operating system versions, leading to potential data quality problems.`consistency`

In [91]:
tem_df['OpSys'].unique()

array(['macOS', 'No OS', 'Windows 10', 'Mac OS X', nan, 'Linux',
       'Windows 10 S', 'Chrome OS', 'Windows 7', 'Android'], dtype=object)

#### Explore the 'Weight' column and identify any inconsistencies

In [92]:
k = tem_df['Weight'].str.split('kg').str.get(0).dropna().str.replace('?','1.1111').unique()

In [93]:
print(sorted(k))

['0.0002', '0.69', '0.81', '0.91', '0.92', '0.920', '0.97', '0.98', '0.99', '1.05', '1.08', '1.09', '1.1', '1.10', '1.11', '1.1111', '1.12', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '1.2', '1.21', '1.22', '1.23', '1.24', '1.25', '1.252', '1.26', '1.27', '1.28', '1.29', '1.3', '1.31', '1.32', '1.34', '1.35', '1.36', '1.37', '1.38', '1.39', '1.4', '1.41', '1.42', '1.43', '1.44', '1.45', '1.47', '1.48', '1.49', '1.5', '1.54', '1.55', '1.56', '1.58', '1.59', '1.6', '1.62', '1.63', '1.64', '1.65', '1.68', '1.7', '1.70', '1.71', '1.74', '1.75', '1.76', '1.78', '1.79', '1.8', '1.83', '1.84', '1.85', '1.86', '1.87', '1.88', '1.89', '1.9', '1.90', '1.91', '1.93', '1.94', '1.95', '1.96', '1.98', '1.99', '11.1', '2', '2.0', '2.02', '2.03', '2.04', '2.05', '2.06', '2.07', '2.08', '2.09', '2.1', '2.13', '2.14', '2.15', '2.16', '2.17', '2.18', '2.19', '2.191', '2.2', '2.20', '2.21', '2.23', '2.24', '2.25', '2.26', '2.29', '2.3', '2.31', '2.32', '2.33', '2.34', '2.36', '2.37', '2.38', 

In [94]:
# this values might be 2kg or 2.2 or 2.02 kg
tem_df[tem_df['Weight'] == '0.0002kg']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
349,Dell,Ultrabook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,1TB HDD,AMD Radeon 530,Windows 10,0.0002kg,35324.64


In [95]:
tem_df[(tem_df['Company'] == 'Dell') & (tem_df['Inches'] == '15.6') & (tem_df['Gpu'] == 'AMD Radeon 530')]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
27,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,42624.0000
72,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,46353.6000
180,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,256GB SSD + 2TB HDD,AMD Radeon 530,Linux,2.02kg,55890.7200
245,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.2kg,52480.8000
315,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,128GB SSD + 1TB HDD,AMD Radeon 530,Windows 10,2.33kg,44968.3200
349,Dell,Ultrabook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,1TB HDD,AMD Radeon 530,Windows 10,0.0002kg,35324.6400
355,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.02kg,51729.5520
426,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,256GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.02kg,60867.0720
441,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,4GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,39906.7200
454,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 1TB HDD,AMD Radeon 530,Windows 10,2.36kg,52161.1200


In [96]:
# correct weight
tem_df[tem_df['Weight'] == '0.69kg']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
1082,Lenovo,2 in 1 Convertible,10.1,IPS Panel Touchscreen 1920x1200,Intel Atom x5-Z8550 1.44GHz,4GB,64GB Flash Storage,Intel HD Graphics 400,Windows 10,0.69kg,34433.2656
1114,Lenovo,2 in 1 Convertible,10.1,IPS Panel Touchscreen 1920x1200,Intel Atom x5-Z8550 1.44GHz,4GB,64GB Flash Storage,Intel HD Graphics 400,Android,0.69kg,29250.7200
1126,Lenovo,2 in 1 Convertible,10.1,IPS Panel Touchscreen 1920x1200,Intel Atom x5-Z8550 1.44GHz,4GB,64GB Flash Storage,Intel HD Graphics 400,Windows 10,0.69kg,25521.1200


In [97]:
tem_df[tem_df['Weight'] == '0.92kg']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
14,Apple,Ultrabook,12,IPS Panel Retina Display 2304x1440,Intel Core M m3 1.2GHz,8GB,256GB SSD,Intel HD Graphics 615,macOS,0.92kg,67260.672
81,Apple,Ultrabook,12,IPS Panel Retina Display 2304x1440,Intel Core i5 1.3GHz,8GB,512GB SSD,Intel HD Graphics 615,macOS,0.92kg,80452.800


In [98]:
pd.set_option('display.max_rows', 200)

In [99]:
print(tem_df.dropna()[tem_df['Weight'].str.split('kg').str.get(0).dropna().str.replace('?','1.1111') >= '4.00']['Weight'].unique().tolist())

['7.2kg', '5.4kg', '4.4kg', '4.42kg', '4.7kg', '4.3kg', '4.14kg', '4.1kg', '5.8kg', '8.23kg', '4.33kg', '4.6kg', '4.5kg', '8.4kg', '6.2kg', '4.36kg', '4.2kg']


- in ascer 7.2kg need convernt to 2.1kg
- ascer 8.23kg to 2.2
- in toshiba 5.4kg need to convert 1.39
- 7.22 lenvo to 2.2 kg
- 8.4 lenovo to 1.27kg

In [100]:
print(tem_df.dropna()[tem_df['Weight'].str.split('kg').str.get(0).dropna().str.replace('?','1.1111') >= '7.00']['Weight'].unique().tolist())

['7.2kg', '8.23kg', '8.4kg']


In [101]:
tem_df[tem_df['Weight'].isin(['7.2kg', '8.23kg', '8.4kg'])]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
133,Acer,Notebook,15.6,Full HD 1920x1080,Intel Core i3 7130U 2.7GHz,4GB,256GB SSD,Nvidia GeForce MX130,Windows 10,7.2kg,30476.16
240,Lenovo,Notebook,15.6,1366x768,Intel Core i3 6006U 2GHz,8GB,128GB SSD,Intel HD Graphics 520,Windows 10,7.2kg,31381.92
326,Acer,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,6GB,1TB HDD,Intel HD Graphics 620,Windows 10,8.23kg,29250.72
587,Lenovo,Ultrabook,14,IPS Panel Full HD 1920x1080,Intel Core i7 6600U 2.6GHz,12GB,256GB SSD,Intel HD Graphics 520,Windows 10,8.4kg,89137.44


- Explore Price column

In [102]:
tem_df['Price'].describe()

count      1273.000000
mean      59955.814073
std       37332.251005
min        9270.720000
25%       31914.720000
50%       52161.120000
75%       79333.387200
max      324954.720000
Name: Price, dtype: float64

In [103]:
tem_df[tem_df['Price'] == 9270.720000]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
1215,Acer,Netbook,11.6,1366x768,Intel Celeron Dual Core 3205U 1.5GHz,2GB,32GB SSD,Intel HD Graphics,Chrome OS,1.3kg,9270.72


# Data Cleaning Order
- Quality -> Completeness
- Structural -> Messy
- Quality -> Validity
- Quality -> Accuracy
- Quality -> Consistency

In [104]:
# first five rows
tem_df.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [105]:
# information of dataframe
tem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           1273 non-null   object 
 1   TypeName          1273 non-null   object 
 2   Inches            1273 non-null   object 
 3   ScreenResolution  1273 non-null   object 
 4   Cpu               1273 non-null   object 
 5   Ram               1273 non-null   object 
 6   Memory            1273 non-null   object 
 7   Gpu               1273 non-null   object 
 8   OpSys             1273 non-null   object 
 9   Weight            1273 non-null   object 
 10  Price             1273 non-null   float64
dtypes: float64(1), object(10)
memory usage: 112.1+ KB


In [106]:
#no of missing values
tem_df.isna().sum()

Company             30
TypeName            30
Inches              30
ScreenResolution    30
Cpu                 30
Ram                 30
Memory              30
Gpu                 30
OpSys               30
Weight              30
Price               30
dtype: int64

In [107]:
# number of duplicated rows
tem_df.duplicated().sum()

np.int64(58)

# Solve Completeness Issues

In [108]:
# shape of dataframe before drop nan rows
tem_df.shape

(1303, 11)

In [109]:
# 30 rows out of 1303 are totally missing along with their 11 columns
# that's why i drop all missing values

tem_df = tem_df.dropna().reset_index(drop=True)

In [110]:
# shape of dataframe after drop nan rows
tem_df.shape

(1273, 11)

In [111]:
# 58 duplicated rows detected, with 30 rows containing all null values which are already droped
# now it's time to drop all duplicated rows
tem_df.drop_duplicates(inplace=True)

In [112]:
# shape of dataframe after drop duplicates rows
tem_df.shape

(1244, 11)

In [113]:
# resolving the '?' entries in the memory column 
# and memory of this particular laptop is 1TB HDD 
tem_df.loc[tem_df['Memory'] == '?', 'Memory'] = '1TB HDD'

# Solve Structural Issues

### **Solve ScreenResolution Structural Problem**

In [114]:
# extract resolution from this column values
# Regular expression pattern to find resolutions
pattern = r'\b\d{3,4}x\d{3,4}\b'

# Find resolutions for each value in the Series and create new column of resoultion
tem_df['screen_resolution'] = tem_df['ScreenResolution'].str.findall(pattern).str.get(0)

In [115]:
# unique display resolution
tem_df['screen_resolution'].unique()

array(['2560x1600', '1440x900', '1920x1080', '2880x1800', '1366x768',
       '2304x1440', '3200x1800', '2256x1504', '3840x2160', '2160x1440',
       '1600x900', '2560x1440', '2736x1824', '2400x1600', '1920x1200'],
      dtype=object)

In [116]:
# create this function so that i can create display type column
def get_display_type(resolution):
    width, height = map(int, resolution.split('x'))
    
    if width >= 3840 and height >= 2160:
        return '4K'
    elif width >= 2560 and height >= 1440:
        return 'Quad HD'
    elif width >= 1920 and height >= 1080:
        return 'Full HD'
    elif width >= 1280 and height >= 720:
        return 'HD'
    else:
        return 'Other'
    
tem_df['display_type'] = tem_df['screen_resolution'].apply(get_display_type)

In [117]:
# frequency of display type
tem_df['display_type'].value_counts()

display_type
Full HD    836
HD         309
Quad HD     57
4K          42
Name: count, dtype: int64

### **Solve Cpu Structural Problem**

In [118]:
# create processor_speed column from Cpu column

# Define regex pattern to match any number followed by "GHz"
pattern = r'\b\d+(?:\.\d+)?(?:GHz|Hz)\b'

# Find all matches in the Processor column
tem_df['processor_speed'] = tem_df['Cpu'].str.findall(pattern).str.get(0).str.split('G').str.get(0)


In [119]:
# unique processor_speed values 
tem_df['processor_speed'].unique()

array(['2.3', '1.8', '2.5', '2.7', '3.1', '3', '2.2', '1.6', '2', '2.8',
       '1.2', '2.9', '2.4', '1.5', '1.9', '1.44', '1.1', '2.0', '1.3',
       '2.6', '3.6', '1.60', '3.2', '1.0', '2.1', '0.9', '1.92', '2.50',
       '2.70'], dtype=object)

In [120]:
# create one more column that is processor_brand
tem_df['processor_brand'] = tem_df['Cpu'].str.split(' ').str.get(0)

In [121]:
# unique processor brand
tem_df['processor_brand'].unique()

array(['Intel', 'AMD', 'Samsung'], dtype=object)

In [122]:
# frequency 
tem_df['processor_brand'].value_counts()

processor_brand
Intel      1183
AMD          60
Samsung       1
Name: count, dtype: int64

In [123]:
tem_df[tem_df['Cpu'].str.contains('Samsung')]

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,screen_resolution,display_type,processor_speed,processor_brand
1163,Samsung,2 in 1 Convertible,12.3,IPS Panel Touchscreen 2400x1600,Samsung Cortex A72&A53 2.0GHz,4GB,32GB Flash Storage,ARM Mali T860 MP4,Chrome OS,1.15kg,35111.52,2400x1600,Full HD,2.0,Samsung


In [124]:
pt = tem_df['Cpu'].str.split('Intel').str.get(1)

# Function to handle unexpected float values
def handle_unexpected_values(val):
    if isinstance(val, list):
        return val[:-1]  # Extract all elements except the last one from the list
    else:
        return []  # Convert unexpected float value to an empty list

pt_list = pt.str.split(' ')

def merge_list_items(lst):
    return ' '.join(lst)

processor_type_intel = pt_list.apply(handle_unexpected_values).apply(merge_list_items)

In [125]:
pa = tem_df['Cpu'].str.split('AMD').str.get(1)

# Function to handle unexpected float values
def handle_unexpected_values(val):
    if isinstance(val, list):
        return val[:-1]  # Extract all elements except the last one from the list
    else:
        return []  # Convert unexpected float value to an empty list

pt_list_amd = pa.str.split(' ')

def merge_list_items(lst):
    return ' '.join(lst)

processor_type_amd = pt_list_amd.apply(handle_unexpected_values).apply(merge_list_items)

In [126]:
sa = tem_df['Cpu'].str.split('Samsung').str.get(1)

# Function to handle unexpected float values
def handle_unexpected_values(val):
    if isinstance(val, list):
        return val[:-1]  # Extract all elements except the last one from the list
    else:
        return []  # Convert unexpected float value to an empty list

pt_list_sa = sa.str.split(' ')

def merge_list_items(lst):
    return ' '.join(lst)

processor_type_sa = pt_list_sa.apply(handle_unexpected_values).apply(merge_list_items)

In [127]:
# create one more column that is processor_type
tem_df['processor_type'] = (processor_type_intel + processor_type_amd + processor_type_sa)

### **Solve Memory Structural Problem**


In [128]:
tem_df['Memory'].str.split('SSD').str.get(0).unique()

array(['128GB ', '128GB Flash Storage', '256GB ', '512GB ', '500GB HDD',
       '256GB Flash Storage', '1TB HDD', '64GB Flash Storage',
       '32GB Flash Storage', '32GB ', '2TB HDD', '64GB ', '1.0TB Hybrid',
       '1TB ', '16GB ', '16GB Flash Storage',
       '64GB Flash Storage +  1TB HDD', '180GB ', '1TB HDD +  1TB HDD',
       '32GB HDD', '512GB Flash Storage', '128GB HDD', '240GB ', '8GB ',
       '508GB Hybrid', '1.0TB HDD'], dtype=object)

In [129]:
tem_df[tem_df['Memory'] == '1TB HDD +  1TB HDD']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,screen_resolution,display_type,processor_speed,processor_brand,processor_type
685,Lenovo,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,1TB HDD + 1TB HDD,Intel HD Graphics 620,Windows 10,2.1kg,33110.856,1920x1080,Full HD,2.5,Intel,Core i5 7200U


In [130]:
tem_df['Memory'].unique()

array(['128GB SSD', '128GB Flash Storage', '256GB SSD', '512GB SSD',
       '500GB HDD', '256GB Flash Storage', '1TB HDD',
       '128GB SSD +  1TB HDD', '256GB SSD +  256GB SSD',
       '64GB Flash Storage', '32GB Flash Storage', '256GB SSD +  1TB HDD',
       '256GB SSD +  2TB HDD', '32GB SSD', '2TB HDD', '64GB SSD',
       '1.0TB Hybrid', '512GB SSD +  1TB HDD', '1TB SSD',
       '256GB SSD +  500GB HDD', '128GB SSD +  2TB HDD',
       '512GB SSD +  512GB SSD', '16GB SSD', '16GB Flash Storage',
       '512GB SSD +  256GB SSD', '512GB SSD +  2TB HDD',
       '64GB Flash Storage +  1TB HDD', '180GB SSD', '1TB HDD +  1TB HDD',
       '32GB HDD', '1TB SSD +  1TB HDD', '512GB Flash Storage',
       '128GB HDD', '240GB SSD', '8GB SSD', '508GB Hybrid', '1.0TB HDD',
       '512GB SSD +  1.0TB Hybrid', '256GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [131]:
pd.set_option('display.max_rows', None)

In [132]:
import re

#create another kf dataframe for create three column from Memory column

data = tem_df['Memory']
kf = pd.DataFrame({'storage':data})

# Function to extract SSD storage size
def extract_ssd_storage(storage):
    ssd_sizes = re.findall(r'(\d+GB) SSD', storage)
    return ' '.join(ssd_sizes)

# Function to extract HDD storage size
def extract_hdd_storage(storage):
    hdd_sizes = re.findall(r'(\d+GB|\d+TB) HDD', storage)
    return ' '.join(hdd_sizes)

# Function to extract flash storage size
def extract_flash_storage(storage):
    flash_sizes = re.findall(r'(\d+GB) Flash Storage', storage)
    return ' '.join(flash_sizes)

# Apply the functions to the 'storage' column
kf['SSD Storage'] = kf['storage'].apply(extract_ssd_storage)
kf['HDD Storage'] = kf['storage'].apply(extract_hdd_storage)
kf['Flash Storage'] = kf['storage'].apply(extract_flash_storage)

# Display the DataFrame with the extracted storage sizes
# kf[['storage', 'SSD Storage', 'HDD Storage', 'Flash Storage']]

In [133]:
# create new(ssd_storage) column from Memory column
tem_df['ssd_storage'] = kf['SSD Storage'].str.split('GB').str.get(0)

# where nan value exists in ssd_storage; i want to assign nan values with '0'
tem_df.loc[tem_df['ssd_storage'] == '', 'ssd_storage'] = '0'

In [134]:
# create another new(hdd_storage) column from Memory column
tem_df['hdd_storage'] = kf['HDD Storage'].str.split('GB').str.get(0)

# where nan value exists in hdd_storage; i want to assign nan values with '0'
tem_df.loc[tem_df['hdd_storage'] == '', 'hdd_storage'] = '0'

In [135]:
# create another new(flash_storage) column from Memory column
tem_df['flash_storage'] = kf['Flash Storage'].str.split('GB').str.get(0)

# where nan value exists in flash_storage; i want to assign nan values with '0'
tem_df.loc[tem_df['flash_storage'] == '', 'flash_storage'] = '0'

In [136]:
tem_df.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,screen_resolution,display_type,processor_speed,processor_brand,processor_type,ssd_storage,hdd_storage,flash_storage
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832,2560x1600,Quad HD,2.3,Intel,Core i5,128,0,0
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232,1440x900,HD,1.8,Intel,Core i5,0,0,128
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000,1920x1080,Full HD,2.5,Intel,Core i5 7200U,256,0,0
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360,2880x1800,Quad HD,2.7,Intel,Core i7,512,0,0
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080,2560x1600,Quad HD,3.1,Intel,Core i5,256,0,0


# Solve Validity Issues

In [137]:
# Upon manually searching using this laptop's specifications, 
# I discovered that the screen size is 15.6 inches.
tem_df[tem_df['Inches'] == '?']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,screen_resolution,display_type,processor_speed,processor_brand,processor_type,ssd_storage,hdd_storage,flash_storage
465,Dell,Workstation,?,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg,128298.24,1920x1080,Full HD,2.8,Intel,Core i7 7700HQ,256,0,0


In [138]:
# so replace this ? with 15.6
tem_df.loc[tem_df['Inches'] == '?', 'Inches'] = '15.6'

In [139]:
# I found a laptop that supposedly weighs 0.0002 kg, which seems improbable.
tem_df[tem_df['Weight'] ==  '0.0002kg']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,screen_resolution,display_type,processor_speed,processor_brand,processor_type,ssd_storage,hdd_storage,flash_storage
339,Dell,Ultrabook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,1TB HDD,AMD Radeon 530,Windows 10,0.0002kg,35324.64,1920x1080,Full HD,1.6,Intel,Core i5 8250U,0,1TB,0


In [140]:
# this values might be 2kg
tem_df.loc[tem_df['Weight'] ==  '0.0002kg', 'Weight'] = '2.0kg'

In [141]:
# wrong value ------->> correct value
# 7.2kg   ----------------> 2.1kg(Acer)
# 8.23kg ----------------> 2.2kg(Acer)
# 5.4kg -----------------> 1.39(toshiba)
# 7.22kg ---------------> 2.2kg(lenovo)
# 8.4kg  ---------------->  1.27kg(lenovo)
# Upon manually searching using this laptop's specifications, 
# and where laptops weights values are validity issues replace those values with accurate weight

tem_df[tem_df['Weight'] == '7.2kg']

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,screen_resolution,display_type,processor_speed,processor_brand,processor_type,ssd_storage,hdd_storage,flash_storage
128,Acer,Notebook,15.6,Full HD 1920x1080,Intel Core i3 7130U 2.7GHz,4GB,256GB SSD,Nvidia GeForce MX130,Windows 10,7.2kg,30476.16,1920x1080,Full HD,2.7,Intel,Core i3 7130U,256,0,0
232,Lenovo,Notebook,15.6,1366x768,Intel Core i3 6006U 2GHz,8GB,128GB SSD,Intel HD Graphics 520,Windows 10,7.2kg,31381.92,1366x768,HD,2,Intel,Core i3 6006U,128,0,0


In [142]:
# assign correct value in 128 row
tem_df.loc[128, 'Weight'] = '2.1kg'

In [143]:
# assign correct value in row 232 
tem_df.loc[232, 'Weight'] = '2.2kg'

In [144]:
tem_df.loc[tem_df['Weight'] == '5.4kg', 'Weight'] = '1.39kg'

In [145]:
tem_df.loc[tem_df['Weight'] == '8.23kg', 'Weight'] = '2.2kg'

In [146]:
tem_df.loc[tem_df['Weight'] == '8.4kg', 'Weight'] = '1.27kg'

# Solve Accuracy Issues


In [147]:
tem_df.loc[88, 'Ram'] = '4GB'

In [148]:
tem_df.loc[611, 'Ram'] = '4GB'

In [149]:
tem_df.loc[tem_df['Ram'] == '1GB', 'Ram'] = '8GB'

In [150]:
## 64GB to 8GB
tem_df.loc[[68, 702], 'Ram'] = '8GB'

# Solve Consistency Issues

In [151]:
# replace all Vero with Acer
tem_df.loc[tem_df['Company'] == 'Vero', 'Company'] = 'Acer'

In [152]:
# Netbook replace with Notebook
tem_df.loc[tem_df['TypeName'] == 'Netbook', 'TypeName'] = 'Notebook'

In [153]:
# change dtype
tem_df['TypeName'] = tem_df['TypeName'].astype('category')

In [154]:
tem_df[tem_df['Inches'] > '19.00']['Inches'].values

array(['25.6', '35.6', '35.6', '27.3', '24', '33.5', '24', '31.6', '35.6'],
      dtype=object)

In [155]:
# some centemeter values have in inches column that's why need to use this func
def cm_to_inches(cm):
    inches = cm / 2.54
    return inches
tem_df.loc[tem_df['Inches'] >= '19.00', 'Inches'] = tem_df[tem_df['Inches'] >= '19.00']['Inches'].astype(float).apply(cm_to_inches)

In [156]:
tem_df['Inches'].astype(float).describe()

count    1244.000000
mean       15.006271
std         1.455059
min         9.448819
25%        14.000000
50%        15.600000
75%        15.600000
max        18.400000
Name: Inches, dtype: float64

In [157]:
# extract only ram without GB
tem_df['ram'] = tem_df['Ram'].str.split('GB').str.get(0).astype(int)

In [158]:
#In the laptop dataset, the 'android OpSys' column exhibits significant inconsistency, 
#prompting its removal from the DataFrame.
tem_df.drop(1086, inplace=True)

In [159]:
tem_df['os'] = tem_df['OpSys'].str.split(' ').str.get(0).str.lower()

In [160]:
tem_df['Weight'] = tem_df['Weight'].str.split('kg').str.get(0)

In [161]:
pd.set_option('display.max_columns', None)

In [162]:
tem_df.columns

Index(['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram',
       'Memory', 'Gpu', 'OpSys', 'Weight', 'Price', 'screen_resolution',
       'display_type', 'processor_speed', 'processor_brand', 'processor_type',
       'ssd_storage', 'hdd_storage', 'flash_storage', 'ram', 'os'],
      dtype='object')

In [163]:
clean_df = tem_df[['Company', 'TypeName', 'Inches','Weight','screen_resolution',
                     'display_type', 'processor_speed', 'processor_brand', 'processor_type',
                     'ssd_storage', 'hdd_storage', 'flash_storage', 'ram', 'os','Price']]

In [164]:
clean_df.head(10)

,Company,TypeName,Inches,Weight,screen_resolution,display_type,processor_speed,processor_brand,processor_type,ssd_storage,hdd_storage,flash_storage,ram,os,Price
0,Apple,Ultrabook,13.3,1.37,2560x1600,Quad HD,2.3,Intel,Core i5,128,0,0,8,macos,71378.6832
1,Apple,Ultrabook,13.3,1.34,1440x900,HD,1.8,Intel,Core i5,0,0,128,8,macos,47895.5232
2,HP,Notebook,15.6,1.86,1920x1080,Full HD,2.5,Intel,Core i5 7200U,256,0,0,8,no,30636.0000
3,Apple,Ultrabook,15.4,1.83,2880x1800,Quad HD,2.7,Intel,Core i7,512,0,0,16,macos,135195.3360
4,Apple,Ultrabook,13.3,1.37,2560x1600,Quad HD,3.1,Intel,Core i5,256,0,0,8,macos,96095.8080
5,Acer,Notebook,15.6,2.1,1366x768,HD,3,AMD,A9-Series 9420,0,500,0,4,windows,21312.0000
6,Apple,Ultrabook,15.4,2.04,2880x1800,Quad HD,2.2,Intel,Core i7,0,0,256,16,mac,114017.6016
7,Apple,Ultrabook,13.3,1.34,1440x900,HD,1.8,Intel,Core i5,0,0,256,8,macos,61735.5360
8,Asus,Ultrabook,14,1.3,1920x1080,Full HD,1.8,Intel,Core i7 8550U,512,0,0,16,windows,79653.6000
9,Acer,Ultrabook,14,1.6,1920x1080,Full HD,1.6,Intel,Core i5 8250U,256,0,0,8,windows,41025.6000


In [165]:
clean_df.to_csv('laptop_clean_dataset.csv', index=False)

### ***I've completed data cleaning for an uncleaned laptop price dataset. If you found my notebook helpful, I'd greatly appreciate an upvote. Feel free to leave any suggestions or feedback in the comment section. Thank you!***